We compute in this worksheet the orbifold fundamental group of another one of the groups of Proposition 4.3 in the paper `Torsion divisors of plane curves with maximal flexes and Zariski pairs` by E. Artal, S. Bannai, T. Shirane and H. Tokunaga. As for other groups we use the package `sirocco` by M.Á Marco and M. Rodríguez. Since we need to know which are the meridians of the irreducible components (including the line at infinity), instead of using the method `fundamental_group` we need to use internal functions of `sirocco` in order to obtain the braid monodromy for a projection. In order to do that, we need to provide a geometric basis of the fundamental group of the complement of the discriminant of the projection, follwoing the ideas of C. Alquézar. A complete version of this construction will be part of `sirocco` in forthcoming versions. At this point we will consider a cell decomposition of a disc containing the discriminant and such that each $2$-cell contains exactly one point of the discriminant. The function `get_voronoi` provides this decomposition.

In [1]:
from sage.schemes.curves import zariski_vankampen as zv

import numpy as np

from matplotlib import pyplot as plt

from scipy.spatial import Voronoi, voronoi_plot_2d

import copy

def get_voronoi(points):
    discpoints = np.array([(CC(a).real(), CC(a).imag()) for a in points])
    added_points = 3 * abs(discpoints).max() + 1.0
    configuration = np.vstack([discpoints, np.array([[added_points, 0], [-added_points, 0],
                                               [0, added_points], [0, -added_points]])])
    V = Voronoi(configuration)
    return (V)

We introduce the curve. The equations are in $\mathbb{Q}$; $f$ is the cubic, $T_1$ is the equation of one triangle (which contains the line at infinity) and $T_2$ is the equation of another triangle (this polynomial has three linear factors in an extension of $\mathbb{Q}$).

In [2]:
S1.<x,y>=QQ[]
f=x^2*y+y^2+x
T1=x*y
T2=x^3 - 3*x*y^2 + y^3 - 3*x^2 - 3*x*y - 3*y + 1
F=(f*T1*T2)(x=x+y)

The variable `disc` contains the points in the discriminant and the variable `segs` the edges where the braids needed to obtain the fundamental group are computed. The variable `V` contains the information of the Voronoi diagram used to compute the braids, which will be developed in the next cells.

In [4]:
disc = zv.discrim(F)
segs = zv.segments(disc)
V=get_voronoi(disc)
#vtc=list(Set(flatten(segs)))
#ars=[tuple([vtc.index(a),vtc.index(b)]) for a,b in segs]

This is a technical cell where the main objects are the following ones:

- `Vre`: It is a dictionnary containing the $2$-cells, as lists of integers denoting the oriented edges. Not all the cells are counterclockwisely oriented. It will be corrected later.
- `borde_e`: It is a list of edges codifying the counterclockwise boundary of the disk.

In [5]:
Ve0=[tuple(_) for _ in V.ridge_vertices if -1 not in _]
Ve={j:Ve0[j-1] for j in [1..len(Ve0)]}
for j in [1..len(Ve0)]:
    Ve[-j]=(Ve[j][1],Ve[j][0])
eV={j:i for i,j in Ve.items()}
Vr0=[_ for _ in V.regions if -1 not in _ and len(_)>0]
Vrv={j:tuple(Vr0[j]) for j in range(len(Vr0))}
aux=Vrv.items()
Rv={_[1]:_[0] for _ in aux}
def vr2ve(reg,cerrado=True):
    res=[]
    n=len(reg)
    for j in range(n-1):
        a=(reg[j],reg[j+1])
        m=eV[a]
        res.append(m)
    if cerrado: 
        m=eV[(reg[n-1],reg[0])]
        res.append(m)
    return (tuple(res))

def ve2vr(reg,cerrado=True):
    if len(reg)==0:
        Print("Error")
        return (None)
    if cerrado and Ve[reg[0]][0]!=Ve[reg[-1]][1]:
        Print("Error")
        return (None)
    for j in range(len(reg)-1):
        if Ve[reg[j]][1]!=Ve[reg[j+1]][0]:
            Print ("Error")
            return (None)
    res = tuple([Ve[a][0] for a in reg])
    if not cerrado:
        res.append(Ve[-1][1])
    return (res)
Vre={j:vr2ve(Vrv[j]) for j in Vrv.keys()}
Vretot=[abs(i) for j in Vre.keys() for i in Vre[j] ]
borde_e_unordered=[j for j in Vretot if Vretot.count(j)==1]
borde_v_unordered=list(Set(flatten([Ve[_] for _ in borde_e_unordered])))
borde_e=[]
borde_v=[0]
a=0
borde_v_unordered.remove(a)
while borde_v_unordered!=[]:
    u=[_ for _ in borde_e_unordered if a in Ve[_]]
    b=u[0]
    borde_e_unordered.remove(b)
    ar=list(copy.copy(Ve[b]))  
    if a==ar[0]:
        borde_e.append(b)
    else:
        borde_e.append(-b)
    ar.remove(a)
    a=ar[0]
    borde_v.append(a)
    if a in borde_v_unordered:
        borde_v_unordered.remove(a)
b=borde_e_unordered[0]
ar=Ve[b]
if a==ar[0]:
    borde_e.append(b)
else:
    borde_e.append(-b)
borde_e=tuple(borde_e)
borde_v=tuple(borde_v)
if borde_v.index(3)>borde_v.index(1):
    borde_v=tuple([0]+[_ for _ in reversed(borde_v[1:])])
    borde_e=tuple([-_ for _ in reversed(borde_e)])

Some normalizations are doing; in particular, all the cells are now positively oriented.

In [6]:
def desplazar(i,lista):
    return (lista[i:]+lista[:i])
borde_e=desplazar(10,borde_e)
guardar=[13, 12, 14, 2, 5, 3, 4, 6, 9, 1, 8, 7, 10, 0, 11]
Vre[13]=desplazar(2,Vre[13])
Vre[12]=desplazar(2,[-i for i in reversed(Vre[12])])
Vre[14]=desplazar(1,[-i for i in reversed(Vre[14])])
Vre[2]=desplazar(1,[-i for i in reversed(Vre[2])])
Vre[5]=desplazar(3,Vre[5])
Vre[3]=desplazar(2,[-i for i in reversed(Vre[3])])
Vre[4]=desplazar(2,[-i for i in reversed(Vre[4])])
Vre[6]=desplazar(1,Vre[6])
Vre[9]=desplazar(3,Vre[9])
Vre[1]=desplazar(2,[-i for i in reversed(Vre[1])])
Vre[8]=desplazar(3,[-i for i in reversed(Vre[8])])
Vre[7]=desplazar(3,Vre[7])
Vre[10]=desplazar(0,Vre[10])
Vre[0]=desplazar(0,Vre[0])
Vre[11]=desplazar(4,[-i for i in reversed(Vre[11])])

The list determining a geometric basis is given. For each element in `caminos`, the first list is a path from the base point to a cell, and the second one a lasso around a cell.

In [8]:
caminos=[([],[43,39,-44,-1])]
caminos+=[(caminos[-1][0]+[43],[40,41,-42,-39])]
caminos+=[(caminos[-1][0]+[40],[-45,15,46,-41])]
caminos+=[(caminos[-1][0]+[-45],[-13,12,14,-15])]
caminos+=[(caminos[-1][0]+[-13],[23,21,17,-25,-24,-12])]
caminos+=[(caminos[-1][0]+[23,21],[16,-18,-17])]
caminos+=[(caminos[-2][0]+[23],[-20,19,4,22,-16,-21])]
caminos+=[(caminos[-1][0]+[-20,19,4],[-6,-26,27,25,18,-22])]
caminos+=[(caminos[-2][0]+[-20],[-34,2,35,10,32,29,-36,-5,-19])]
caminos+=[(caminos[-1][0]+[-34,2,35],[9,-11,-10])]
caminos+=[(caminos[-1][0]+[9],[33,-28,-32,11])]
caminos+=[(caminos[-1][0]+[33],[31,-30,-29,28])]
caminos+=[(caminos[-1][0]+[31],[-37,-8,36,30])]
caminos+=[(caminos[-1][0]+[-37],[-7,6,-4,5,8])]
caminos+=[(caminos[-6][0]+[-34,2,],[-3,38,26,7,37,-31,-33,-9,-35])]

We check each cell is represented by an element of `caminos`.

In [9]:
coincide=True
for j in range(15):
    coincide=coincide and (caminos[j][1]==[i for i in Vre[guardar[j]]])
coincide

True

The package `sirocco` is used to obtain the braids.

In [11]:
tr1=[zv.braid_in_segment(F,*s) for s in segs]

We define a free group generated by the edges and we translate the paths in `caminos` as elements in this group, and also the boundary of the union of cells. We check that it is a geometric basis as the reversed product equals the boundary.

In [15]:
FG=FreeGroup(len(Ve0))
base1=[]
for u in caminos:
    gen1=[FG(a) for a in u]
    base1.append(gen1)
infinito=FG(borde_e)
len(base1)

15

In [16]:
lazos=[a[0]*a[1]/a[0] for a in base1]
infinito==prod(_ for _ in reversed(lazos))

True

We define a morphism from this free group to the braid group to construct the braid monodromy

In [18]:
B=BraidGroup(8)
libre2braid=FG.hom(tr1,B)
trenzas=[libre2braid(a) for a in lazos]
[_.exponent_sum() for _ in trenzas]

[2, 8, 1, 2, 8, 2, 1, 1, 1, 2, 8, 2, 2, 8, 1]

We want to modify the above decomposition in order to have positive braids for the middle braids. The result will be kept in the list `puiseux`. Each element of the list `puiseux` have four entries: two braids $\tau_1,\tau_2$ ($\tau_2$ is a positive algebraic braid) and two numbers $m,n$. The braid associated to the basis element is $\tau_1\cdot\tau_2\cdot\tau_1^{-1}$; $m$ is the first strand involved in $\tau_2$ and $n$ is the number of strands involved in $\tau_2$.

In order to keep this computations as short as possible, we need to manipulate the braids one by one.

In [234]:
puiseux=[]

In [235]:
trenzas[0]==B((7, 7))

True

In [236]:
puiseux.append([B.one(),B([7,7]),7,2])

In [237]:
trenzas[1]==B((7, 6, 5, 6, 5, 6, 5, 6, 6, -7))

True

In [238]:
puiseux.append([B([7]),B([6, 5, 6, 5, 6, 5, 6, 6]),5,3])

In [239]:
u=B((7, 6, 6 ,5))
v=B((4,))
trenzas[2]==u*v/u

True

In [240]:
puiseux.append([u,v,4,2])

In [241]:
u=B((7, 6 ,5, 4))
v=B((3,))^2
trenzas[3]==u*v/u

True

In [242]:
puiseux.append([u,v,3,2])

In [243]:
u=B((7, 6 ,5, 4, 3))
v=B((1, 2, 1, 2, 1, 2, 1, 1))
trenzas[4]==u*v/u

True

In [244]:
puiseux.append([u,v,1,3])

In [245]:
u=B((5, ))
v=B((6,))^2
trenzas[5]==u*v/u

True

In [246]:
puiseux.append([u,v,6,2])

In [247]:
u=B([1, 1, 2])
v=B((3,))
trenzas[6]==u*v/u

True

In [248]:
puiseux.append([u,v,3,2])

In [249]:
u=B([5, -3, 4])
v=B((3,))
trenzas[7]==u*v/u

True

In [250]:
puiseux.append([u,v,3,2])

In [251]:
u=B([-6, 4, 5])
v=B((6,))
trenzas[8]==u*v/u

True

In [252]:
puiseux.append([u,v,6,2])

In [253]:
u=B([1])
v=B((2,))^2
trenzas[9]==u*v/u

True

In [254]:
puiseux.append([u,v,2,2])

In [255]:
u=B([1, 2, -3, -6, -6])
v=B(2*(5, 4, 5)+ 2*(5,))
trenzas[10]==u*v/u

True

In [256]:
puiseux.append([u,v,4,3])

In [257]:
u=B([1, -6, -6, -6, -5, 4, 2,])
v=B((3,))^2
trenzas[11]==u*v/u

True

In [258]:
puiseux.append([u,v,3,2])

In [259]:
u=B([-6, 5, -3, 6,  4, -5])
v= B((6, ))^2
trenzas[12]==u*v/u

True

In [260]:
puiseux.append([u,v,6,2])

In [261]:
u=B([4, 7, 6, 5, 4, 6, 7, -6, -6, 5, -6, 3])
v=B(2*(5, 4, 5)+ 2*(4,))
trenzas[13]==u*v/u

True

In [262]:
puiseux.append([u,v,4,3])

In [263]:
u=B([-6, -6, 5, 4, 3])
v=B([2])
trenzas[14]==u*v/u

True

In [264]:
puiseux.append([u,v,2,2])
len(puiseux)

15

We have all the Puiseux decompositions of the braids. We need to identify to which component each meridian belongs. We see that the generators $\mu_2,\mu_4,\mu_6$ correspond to the cubic. The other ones (and the meridian at infinity) correspond to the lines. We may distinguish the two triangles but we will do it later. This is done looking at the orbits by the monodromy action.

We start with the free group corresponding to the fundamental group of the complement of the curve in a vertical line. We add the orbifold relations: the cubes of the meridians of the lines and the 9th power of the meridian of the cubic.

In [265]:
permutaciones=PermutationGroup([_.permutation() for _ in trenzas])
print(permutaciones.orbits())
FL8=FreeGroup(8)
L8=[FL8(_)^3 for _ in [[1], [3], [5], [7], [8], [1..8]]]+[FL8([2])^9]

[[1], [2, 4, 6], [3], [5], [7], [8]]


We add the relations of the braid monodromy using the Puiseux decomposition.

In [266]:
for tau1,tau2,m0,m1 in puiseux:
    for j in [m0..m0+m1-1]:
        v=((FL8([j])*tau2)/FL8([j]))*tau1^-1
        L8.append(v)

In [267]:
G=(FL8/L8)

We simplify the presentation and we keep track of the meridians. We see that this orbifold group is finite. As it is bigger than the abelianization, the group is not abelian.

In [268]:
hom1=G.simplification_isomorphism()
G1=hom1.codomain()

In [269]:
G1.order()

6561

In [270]:
prod(G1.abelian_invariants())

2187

We express the meridians in terms of the generators of $G_1$.

In [273]:
meridianos=[hom1(G(_)).Tietze() for _ in [[1],[3],[5],[7],[8],[-8..-1],[2]]]
meridianos

[(1,),
 (3,),
 (4,),
 (5,),
 (6,),
 (-6, -5, 3, 1, -2, -1, -3, -4, 1, -2, -1, -3, -2, -1),
 (2,)]

Since there are `GAP` functions which are not natively translated in `Sagemath` we translate some objects into `GAP`.

In [275]:
G1gap=G1.gap()
meridianosgap=[G1(_).gap() for _ in meridianos]
hom=G1gap.IsomorphismPermGroup()
G1a=hom.Range()
[hom.Image(_).Order() for _ in meridianosgap]

[3, 3, 3, 3, 3, 3, 9]

We check the commutators of the meridians. One of them, $v$, is the generator of the derived subgroup.

In [276]:
G1gap.DerivedSubgroup().Size()

3

In [277]:
conms={(i,j):hom.Image(meridianosgap[i]*meridianosgap[j]/meridianosgap[i]/meridianosgap[j]) for i in [0..5] for j in [i+1..6]}
for i in [0..4]:
    for j in [i+1..5]:
        print (i,j,conms[i,j].Order())

0 1 1
0 2 3
0 3 1
0 4 3
0 5 1
1 2 1
1 3 3
1 4 1
1 5 3
2 3 1
2 4 3
2 5 1
3 4 1
3 5 3
4 5 1


In [278]:
v=conms[0,2]

The commutations of the meridians allow us to identify the triangles. The meridians of distinct triangles pairwise commute. And the meridians in the same triangle do not commute.

In [281]:
X=[0,2,4]
Y=[1,3,5]

In [282]:
for i in X:
    for j in X:
        if i<j:
            print(conms[i,j].Order())

3
3
3


In [283]:
for i in Y:
    for j in Y:
        if i<j:
            print(conms[i,j].Order())

3
3
3


In [284]:
for i in X:
    for j in Y:
        if i<j:
            print(conms[i,j].Order())
        if j<i:
            print(conms[j,i].Order())

1
1
1
1
1
1
1
1
1


We check which commutators equal $v$ or $v^{-1}$.

In [285]:
for i in [0..4]:
    for j in [i+1..5]:
        if conms[i,j].Order()==3:
            print(v==conms[i,j])

True
False
False
True
True
False


In [286]:
for i in [0..5]:
    u=conms[i,6]
    if u.Order()==1:
        print(i,"trivial")
    elif u==v:
        print(i,"=v")
    else:
        print(i,"=inverse of v")

0 =inverse of v
1 =inverse of v
2 =inverse of v
3 =inverse of v
4 =inverse of v
5 =inverse of v
